In [3]:
import cv2
import mediapipe as mp
import numpy as np
import math
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
class Workout:
    def __init__(self, standing, squat, stomach, stomachToSquat, back, backToSquat, squatToStand):
        self.standing = standing
        self.squat = squat
        self.stomach = stomach
        self.stomachToSquat = stomachToSquat
        self.back = back
        self.backToSquat = backToSquat
        self.squatToStand = squatToStand

    def test(self):
        print('standing - ', self.standing)
        print('squatStomach - ', self.squat)
        print('stomach - ', self.stomach)
        print('squatBack - ', self.stomachToSquat)
        print('back - ', self.back)
        print('squatStomachEnd - ', self.backToSquat)
        print('stomachStand - ',self.squatToStand)
        print('squatStand - ',self.squatToStand)
        return ' '

In [5]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360 - angle
    
    return angle

In [6]:
def calculate_slope(a,b):
    a = np.array(a)
    b = np.array(b)

    slope = (a[1] - b[1]) / (a[0] - b[0])
    
    return slope

In [7]:
def click_event(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        print (x, ' ', y)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, str(x) + ', ' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv2.imshow('image', image)

In [8]:
# def check_starting_pose parbauda, ka kermenis ir taisnam rokam, kajam
def check_starting_pose(angleLeftLeg, angleLeftArm, angleLeftShoulder, angleRightLeg, angleRightArm, angleRightShoulder):
    
    if (angleLeftLeg > 150 and angleLeftArm > 150 and angleLeftShoulder > 150) or (angleRightLeg > 150 and angleRightArm > 150 and angleRightShoulder > 150):
        return True
    return False

In [9]:
def check_squat_pose(angleLeftHip, angleRightHip):

    if angleLeftHip < 60 or angleRightHip < 60:
        return True
    return False

In [10]:
def lay_pose(slopeLeftHeelWrist, angleLeftLeg, angleLeftArm, angleLeftShoulder, slopeRightHeelWrist, angleRightLeg, angleRightArm, angleRightShoulder):
    if (((slopeLeftHeelWrist > 0 and slopeLeftHeelWrist < 2)) and (angleLeftLeg > 150 and angleLeftArm > 150 and angleLeftShoulder > 150)) or (((slopeRightHeelWrist > 0 and slopeRightHeelWrist < 2)) and (angleRightLeg > 150 and angleRightArm > 150 and angleRightShoulder > 150)):
        return True
    return False

In [12]:
#Workout complex
cap = cv2.VideoCapture('c:/test-atrs.mp4')

counter = 0
workout = Workout(False, False, False, False, False, False, False, False)

#set up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose: 
    while cap.isOpened():
        ret, frame = cap.read()

        #re-render image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #make detection
        results = pose.process(image)

        #re-render image to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        #Extract joint landmark coordinates
        try:
            landmarks = results.pose_landmarks.landmark

            #gets joint coordinate points
            leftHip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            leftKnee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            leftAnkle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            leftHeel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]

            rightHip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            rightKnee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            rightAnkle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            rightHeel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]

            rightShoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            rightElbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            rightWrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            leftShoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            leftElbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            leftWrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            #calculate angles
            angleLeftLeg = calculate_angle(leftHip, leftKnee, leftAnkle)
            angleLeftArm = calculate_angle(leftShoulder, leftElbow, leftWrist)
            angleLeftShoulder = calculate_angle(leftElbow, leftShoulder, leftHip)

            angleRightLeg = calculate_angle(rightHip, rightKnee, rightAnkle)
            angleRightArm = calculate_angle(rightShoulder, rightElbow, rightWrist)
            angleRightShoulder = calculate_angle(rightElbow, rightShoulder, rightHip)

            angleLeftHip = calculate_angle(leftShoulder, leftHip, leftKnee)
            angleRightHip = calculate_angle(rightShoulder, rightHip, rightKnee)

            #gets slope
            slopeLeftHeelWrist = calculate_slope(leftWrist, leftHeel)
            slopeRightHeelWrist = calculate_slope(rightWrist, rightHeel)
            
            #check if standing pose true
            checkStanding = check_starting_pose(angleLeftLeg, angleLeftArm, angleLeftShoulder, angleRightLeg, angleRightArm, angleRightShoulder)

            if checkStanding == True:
                workout.standing = True
                print('', counter, workout.test())
                
            #check if squat pose true
            checkSquat = check_squat_pose(angleLeftHip, angleRightHip)

            if workout.standing == True and checkSquat == True:
                workout.squatStomach = True
                
            #checks if lays down on stomach
            checkLay = lay_pose(slopeLeftHeelWrist, angleLeftLeg, angleLeftArm, angleLeftShoulder, slopeRightHeelWrist, angleRightLeg, angleRightArm, angleRightShoulder)
            
            if workout.standing == True and workout.squatStomach == True and checkLay == True:
                workout.stomach = True
                
            #squat
            if workout.stomach == True and workout.squatStomach == True and checkSquat == True:
                workout.squatBack = True
                
            #lay down - back
            if workout.stomach == True and workout.squatStomach == True and workout.squatBack == True and checkLay == True:
                workout.back = True
            
            #squat
            if workout.back == True and checkSquat == True:
                workout.squatStomachEnd = True
            
            #lay down - stomach
            if workout.squatStomachEnd == True and checkLay == True:
                workout.stomachStand = True
                
            #squat
            if workout.standing == True and workout.squatStomach == True and workout.stomach == True and workout.squatBack == True and workout.back == True and workout.squatStomachEnd == True and workout.stomachStand == True and checkSquat == True:
                workout.squatStand = True

            if (workout.standing == True and workout.squatStomach == True and workout.stomach == True and workout.squatBack == True and workout.back == True and workout.squatStomachEnd == True and workout.stomachStand == True and workout.squatStand == True):
                counter += 1
                workout.standing = False
                workout.squatStomach = False
                workout.stomach = False
                workout.squatBack = False
                workout.back = False
                workout.squatStomachEnd = False
                workout.stomachStand = False
                workout.squatStand = False
                checkStanding = 0
                checkSquat = 0
                checkLay = 0
        except:
            pass
        
        #render counter
        cv2.rectangle(image, (0,0), (225, 73), (245, 117, 16), -1)

        #counter data
        cv2.putText(image, 'REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        #render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Mediapipe feed', image)
        cv2.setMouseCallback('Mediapipe feed', click_event)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

standing -  True
squatStomach -  False
stomach -  False
squatBack -  False
back -  False
squatStomachEnd -  False
stomachStand -  False
squatStand -  False
 0  
standing -  True
squatStomach -  False
stomach -  False
squatBack -  False
back -  False
squatStomachEnd -  False
stomachStand -  False
squatStand -  False
 0  
standing -  True
squatStomach -  False
stomach -  False
squatBack -  False
back -  False
squatStomachEnd -  False
stomachStand -  False
squatStand -  False
 0  
standing -  True
squatStomach -  False
stomach -  False
squatBack -  False
back -  False
squatStomachEnd -  False
stomachStand -  False
squatStand -  False
 0  
standing -  True
squatStomach -  False
stomach -  False
squatBack -  False
back -  False
squatStomachEnd -  False
stomachStand -  False
squatStand -  False
 0  
standing -  True
squatStomach -  False
stomach -  False
squatBack -  False
back -  False
squatStomachEnd -  False
stomachStand -  False
squatStand -  False
 0  
standing -  True
squatStomach -  F